In [ ]:
#installing required packages
!pip install shap
!pip install catboost
!pip install lime
!pip install interpret

In [ ]:
# Import necessary libraries for data manipulation, machine learning models, and evaluation metrics.
import os
import pandas as pd
import numpy as np
import shap
import logging
import matplotlib
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NearMiss
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer, StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lime.lime_tabular import LimeTabularExplainer
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')
# Suppress warnings to clean output, including deprecation warnings which are common with evolving libraries.
warnings.filterwarnings('ignore')
matplotlib.use('Agg')  # Use Agg backend for matplotlib to enable plot generation without a GUI.

In [ ]:
# Define the path to the dataset. 
path = 'data_renamed.csv'

# Load the dataset into a pandas DataFrame and read the CSV file located at the specified path and load its contents into the DataFrame 'df'.
df = pd.read_csv(path)

# Access the 'stroke' column of the DataFrame. This column contains the target variable we aim to predict.
df['stroke']

In [3]:
class StrokePrediction:
    def __init__(self, data_path, mode):
        # Load the dataset from a specified path.
        self.data = pd.read_csv(data_path,  encoding ='latin1', sep =',')
        #self.data = self.data[:1000]

        # Print the initial shape (number of rows and columns) of the dataset for a quick overview of its size.
        print(self.data.shape)
        # Create a deep copy of the original data for any operations that might require reference to the unmodified dataset.
        self.data_original = self.data.copy()

        # Initialize a dictionary to store models, an empty list for top features selected by the models,
        # and a dictionary to store ROC AUC scores for model evaluation.
        self.models = {}
        self.top_features = []
        self.mode = mode # Mode specifies the preprocessing strategy to use.
        self.roc_auc = {}

        # Handle different data preprocessing strategies based on the 'mode'.
        if self.mode=='undersample':
            # For 'undersample' mode, use NearMiss algorithm to balance the dataset by undersampling the majority class.
            # This can help in cases where the dataset is highly imbalanced towards one class.           
            X = self.data.drop('stroke', axis=1) # Features (exclude 'stroke' column).
            y = self.data['stroke'] # Target variable.

            undersample = NearMiss(version=3, n_neighbors_ver3=3)
            # Transform the dataset
            X, y = undersample.fit_resample(X, y)
            # Print the class distribution after undersampling to verify the balancing.
            a,b = np.unique(y, return_counts=True)
            print(a,b)

        elif self.mode=='smote_nc':
            # For 'smote_nc' mode, use SMOTENC for oversampling, particularly useful when categorical features are present.
            # SMOTENC can handle both continuous and categorical features, addressing class imbalance by generating synthetic samples.
            X = self.data.drop('stroke', axis=1)
            y = self.data['stroke']
            # List of categorical feature names that will be treated specially by SMOTENC.
            cat_features = ['DHH_MS', 'DHH_AGE', 'Working','Working_Last_Week','Regular_Healthcare_Provider', 'Urban_Rural','Household_Ownership','Problem_Handling_Ability',
            'Day_to_Day_Demands_Ability','Source_of_Stress','Perceived_Health','Perceived_Mental_Health','Perceived_Life_Stress','Food_Security', 'Food_Affordability','Sense_of_Community_Belonging',
            'Level_of_Education','Drank_Alcohol_12mo','Drank_Alcohol_freq','Drank_more_than_4_once_freq','Asthma','High_BP','High_BP_took_meds','Diabetes',
            'Heart_Disease','High_Cholesterol_took_meds','Smoked_more_than_100_lifetime','Smoker_Type', 'Self_percieved_weight','Expose_to_smoke_pvt_vehicle',
            'Expose_to_smoke_public', 'Mood_Disorder','Anxiety_Disorder','Mental_Health_Consult_12mo','White','Chinese','South_Asian','Black','Filipino','Latin_America',
            'South_East_Asian','Arab','West_Asian','Japanese','Korean','Canadian','French','English','German','Scottish','Irish','Italian',
            'Ukrainian','Dutch','Chinese_Ethnic','Jewish','Polish','South_Asian_Ethnic','Norwegian', 'Welsh','Swedish','Other','Metis','Inuit']
            
            smote_nc = SMOTENC(categorical_features=cat_features, random_state=0)
            X, y = smote_nc.fit_resample(X, y)
        else:
            #data['DHH_SEX'] = data['DHH_SEX'].map({'Male': 0, 'Female': 1})
            # Default preprocessing involves handling missing values using KNNImputer.
            # This is particularly useful in datasets where dropping rows with missing values might lead to significant data loss.
            imputer = KNNImputer(n_neighbors=5) # Imputes missing values based on the 5 nearest neighbors.
            transformed_data = imputer.fit_transform(self.data)
            # Replace the original dataset with the imputed one, maintaining the same structure and column names.
            self.data = pd.DataFrame(transformed_data, columns=self.data.columns)
            X = self.data.drop('stroke', axis=1) # Features
            y = self.data['stroke'] # Target variable

        # Split the dataset into training and testing sets, with 20% of the data reserved for testing.
        # Stratify parameter ensures that the proportion of classes in the dataset is preserved in both training and testing sets.
        # This is crucial for maintaining a representative distribution of classes, especially in imbalanced datasets.
        #self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
        #preprocessed_data = pd.concat([X, pd.Series(y, name='stroke')], axis=1)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


    def explain_with_lime(self, model, X_train, X_test, model_name):
        # Initialize the LIME explainer with the training data. This explainer will be used to generate explanations
        # for predictions made by the model, providing insights into which features contribute most to each prediction.
        explainer = LimeTabularExplainer(
            training_data=self.X_train.values,  # The training dataset used for model fitting.
            feature_names=self.X_train.columns,  # Names of the features in the dataset for interpretability.
            class_names=['No Stroke', 'Stroke'],  # The outcome classes for the binary classification task.
            verbose=True,  # Enables detailed logging of the explanation generation process.
            mode='classification'  # Specifies that the model is being used for classification.
        )

        # Select a random instance from the test dataset to explain. This helps in understanding how the model makes predictions for individual data points.
        instance = self.X_test.sample(n=1).values[0]

        # Generate an explanation for the selected instance using LIME. This explanation will highlight the top features that influence the model's prediction for this particular data point.
        exp = explainer.explain_instance(
            data_row=instance,  # The data point to explain.
            predict_fn=model.predict_proba,  # The prediction function of the model (probability outputs required).
            num_features=5  # Limit the explanation to the top 5 features for clarity.
        )

        # Generate a visual plot of the explanation with matplotlib, showing the contribution of each feature to the model's prediction.
        fig = exp.as_pyplot_figure()

        # Predict the class for the selected instance to add context to the explanation plot.
        predicted_class = model.predict(instance.reshape(1, -1))[0]
        predicted_class_name = 'Stroke' if predicted_class else 'No Stroke'

        # Customize the plot for better readability and understanding.
        fig.set_size_inches(10, 6)  # Adjust figure size for visibility.
        plt.xlabel('Important Features')  # Label for the x-axis.
        plt.ylabel('Contribution to Prediction')  # Label for the y-axis.
        plt.title(f"Visualization of the local contribution of features in classifying a single test instance (predicted class = {predicted_class_name}) using the {model_name} model.")
        plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability.
        plt.tight_layout()  # Adjust the layout to make sure everything fits without overlap.

        # Save the generated plot to a file. 
        filename = f"{model_name}_lime_explanation.png"
        plt.savefig(filename)
        plt.show()  # Display the plot in the notebook


    def plot_feature_importance(self, model, feature_names, model_name):
        # Retrieve the feature importances from the model. This assumes the model has a `feature_importances_` attribute,
        # common in tree-based models like RandomForest, GradientBoosting, and XGBoost.
        importances = model.feature_importances_
        # Sort the indices of the importances in ascending order and select the top 15 features for visualization.
        indices = np.argsort(importances)[-15:]
        plt.figure(figsize=(10, 7)) # Adjust figure size for visibility.
        plt.title(f"Top 15 Feature Importances for {model_name}")
        # Create a horizontal bar chart to visualize the relative importance of the top 15 features.
        # The y-axis is the range of the number of features, and the x-axis represents their relative importance.   
        plt.barh(range(len(indices)), importances[indices], align="center")
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices]) # Set the y-ticks to correspond to the feature names, using the indices to map back to the original names.
        plt.xlabel("Relative Importance") # Label for the x-axis.

        # Save the generated plot to a file. 
        filename = f"{model_name}_feature_importance.png"
        plt.savefig(filename)
        plt.show() # Display the plot in the notebook


    def train_model(self):
        # Define a dictionary mapping model names to their respective classifier instances.
        # This approach allows for easy expansion or modification of the model lineup.
        model_definitions = {
            'RandomForest': RandomForestClassifier(),  # Ensemble method based on decision trees.
            'GradientBoosting': GradientBoostingClassifier(),  # Boosting method for high performance.
            'KNN': KNeighborsClassifier(),  # A simple, distance-based classifier.
            'NeuralNetwork': MLPClassifier(max_iter=1000),  # Multi-layer Perceptron classifier.
            'LGBM': LGBMClassifier(),  # LightGBM model, efficient for large datasets and high performance.
            'CatBoost': CatBoostClassifier(verbose=0),  # CatBoost, optimized for categorical data.
            'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # XGBoost, known for performance and flexibility.
        }

        # Define a dictionary to hold the hyperparameters grids for each model. 
        # These grids specify the parameters to be explored during hyperparameter tuning, allowing for systematic experimentation to find the most effective model configurations.
        hyperparameters = {
            'RandomForest': {
                # Parameters for RandomForestClassifier, specifying the number of trees (n_estimators),
                # the maximum depth of the trees (max_depth), and the minimum number of samples required 
                # to split an internal node (min_samples_split) and to be at a leaf node (min_samples_leaf).
                'randomforestclassifier__n_estimators': [100, 200],
                'randomforestclassifier__max_depth': [10, 20],
                'randomforestclassifier__min_samples_split': [2, 5],
                'randomforestclassifier__min_samples_leaf': [1, 2]
            },
            'GradientBoosting': {
                # Parameters for GradientBoostingClassifier, including the number of boosting stages (n_estimators),
                # the learning rate which scales the contribution of each tree (learning_rate), 
                # and the maximum depth of the regression estimators (max_depth).
                'gradientboostingclassifier__n_estimators': [100, 200],
                'gradientboostingclassifier__learning_rate': [0.01, 0.05],
                'gradientboostingclassifier__max_depth': [3, 4]
            },
            'KNN': {
                # Parameters for KNeighborsClassifier, specifying the number of neighbors to use (n_neighbors),
                # the weight function used in prediction (weights), and the power parameter for the Minkowski metric (p).
                'kneighborsclassifier__n_neighbors': [3, 5],
                'kneighborsclassifier__weights': ['uniform', 'distance'],
                'kneighborsclassifier__p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
            },
            'NeuralNetwork': {
                # Parameters for MLPClassifier (Neural Network), including the size of the hidden layers (hidden_layer_sizes),
                # the activation function for the hidden layers (activation), the L2 penalty (regularization term) parameter (alpha),
                # and the learning rate schedule (learning_rate).
                'mlpclassifier__hidden_layer_sizes': [(50, 50)],
                'mlpclassifier__activation': ['relu'],
                'mlpclassifier__alpha': [0.0001, 0.001],
                'mlpclassifier__learning_rate': ['constant']
            },
            'LGBM': {
                # Parameters for LGBMClassifier, specifying the number of boosting iterations (n_estimators),
                # the learning rate (learning_rate), and the maximum depth of the trees (max_depth).
                'lgbmclassifier__n_estimators': [100, 200],
                'lgbmclassifier__learning_rate': [0.01, 0.05],
                'lgbmclassifier__max_depth': [3, 4]
            },
            'CatBoost': {
                # Parameters for CatBoostClassifier, defining the number of training iterations (iterations)
                # and the learning rate (learning_rate).
                'catboostclassifier__iterations': [500],
                'catboostclassifier__learning_rate': [0.01, 0.05]
            },
            'XGBoost': {
                # Parameters for XGBClassifier, including the number of gradient boosted trees (n_estimators),
                # the learning rate (learning_rate), and the maximum depth of the trees (max_depth).
                'xgbclassifier__n_estimators': [100, 200],
                'xgbclassifier__learning_rate': [0.01, 0.05],
                'xgbclassifier__max_depth': [3, 4]
            }
        }

        # Initialize an empty dictionary to store the scores or performance metrics of the models 
        # after hyperparameter tuning. This will facilitate comparison and selection of the best performing model.
        scores = {}


        # Iterate over each model defined in model_definitions.
        for name, model in model_definitions.items():
            # Log the start of training for the current model.
            logging.info(f"Training {name}...")
            
            # Create a pipeline with standard scaling followed by the model. StandardScaler standardizes features
            # by removing the mean and scaling to unit variance, which is often beneficial for many models.
            pipeline = make_pipeline(StandardScaler(), model)
            
            # Initialize GridSearchCV with the model's pipeline and its specific hyperparameters.
            # StratifiedKFold is used for cross-validation to ensure each fold is a good representative of the whole,
            # with n_splits=2 for simplicity and faster computation.
            grid_search = GridSearchCV(pipeline, hyperparameters[name], cv=StratifiedKFold(n_splits=2))
            
            # Fit GridSearchCV on the training data to find the best hyperparameters.
            grid_search.fit(self.X_train, self.y_train)
            
            # Store the best score obtained during the hyperparameter tuning for this model.
            scores[name] = grid_search.best_score_
            
            # Save the best performing model to the models dictionary for later use.
            self.models[name] = grid_search.best_estimator_
            
            # Log the best score and parameters for the current model.
            logging.info(f"{name} Best Score: {grid_search.best_score_}")
            logging.info(f"{name} Best Params: {grid_search.best_params_}")


        # After tuning, iterate over the models to evaluate them.
        for name, model in self.models.items():
            # Log the evaluation process start for the current model.
            logging.info(f"Evaluating model: {name}")

            # Determine the appropriate key name for accessing hyperparameters based on the model's name.
            # This key name adjustment is necessary because the naming convention in the hyperparameters dictionary
            # uses the classifier's class name in lowercase followed by 'classifier', but with exceptions like 'XGBoost'.
            if name == "XGBoost":
                key_name = "xgbclassifier"
            elif name == "KNN":
                key_name = "kneighborsclassifier"
            elif name == "NeuralNetwork":
                key_name = "mlpclassifier"
            else:
                key_name = name.lower() + "classifier"

            # Check if the current model has an attribute 'feature_importances_'.
            if hasattr(model.named_steps[key_name], 'feature_importances_'):
                # Plot the feature importances if the model supports this attribute.
                self.plot_feature_importance(model.named_steps[key_name], self.X_train.columns, name)
                
                # Retrieve the feature importances from the model.
                feature_importances = model.named_steps[key_name].feature_importances_
                
                # Extend the list of top features with the names of the top 10 features based on their importances.
                self.top_features.extend(list(self.X_train.columns[np.argsort(feature_importances)[-10:]]))

                # Retrieve the fitted model from the pipeline for further analysis.
                fitted_model = model.named_steps[key_name]
                
                # Determine the appropriate SHAP explainer based on the model type.
                if isinstance(fitted_model, (RandomForestClassifier, GradientBoostingClassifier, XGBClassifier, LGBMClassifier, CatBoostClassifier, KNeighborsClassifier)):
                    # For tree-based models, use the TreeExplainer.
                    explainer = shap.TreeExplainer(fitted_model)
                elif isinstance(fitted_model, MLPClassifier):
                    # For MLPClassifier, TreeExplainer is used with a subset of training data to approximate.
                    explainer = shap.TreeExplainer(fitted_model, self.X_train.sample(n=100))
                else:
                    # For other model types, use KernelExplainer as a more general approach, again with a sample.
                    explainer = shap.KernelExplainer(fitted_model.predict, self.X_train.sample(n=100))
                
                # Clear the current figure to ensure a clean slate for SHAP visualization.
                plt.clf()

                #explainer = shap.TreeExplainer(fitted_model)
                
                # Compute SHAP values for the training set.
                shap_values = explainer.shap_values(self.X_train)
                
                # Generate and display a summary plot of the SHAP values for all features across all samples in the X_train.
                shap.summary_plot(shap_values, self.X_train, plot_type="bar")
                
                # Save the SHAP summary plot to a file for later reference.
                filename = f"{name}_shap_plot.png"
                plt.savefig(filename)
                plt.show()

            # Evaluate using LIME
            # Invoke the LIME explanation method for the current model to provide interpretable explanations
            # for model predictions on individual instances. This is crucial for understanding how model decisions are made.
            self.explain_with_lime(model, self.X_train, self.X_test, name)

        # After evaluating all models and collecting the top features from each, ensure the list of top features is unique by converting it to a set and back to a list.
        self.top_features = list(set(self.top_features))
        # Print the unique top features for inspection. These features are deemed most important across all models.
        print(self.top_features)

    def evaluate_model(self):
        # Iterate over each model stored in the self.models dictionary.
        for name, model in self.models.items():
            # Generate predictions for the test set using the current model.
            predictions = model.predict(self.X_test)
            # Print a classification report for each model, providing a detailed performance analysis
            # including precision, recall, f1-score, and support for each class.
            print(f"Classification Report ({name}):\n{classification_report(self.y_test, predictions)}")
        
        # Rank the models based on their performance on the test set and select the top 5.
        # This ranking uses the model's score method, which typically defaults to accuracy for classifiers.
        top_models = sorted(self.models.items(), key=lambda x: x[1].score(self.X_test, self.y_test), reverse=True)[:5]
        
        # Create an ensemble of the top 5 models using VotingClassifier with 'hard' voting, which means that the predicted class label will be the class that receives the most votes.
        ensemble = VotingClassifier(estimators=top_models, voting='hard')
        
        # Train the ensemble model using only the top features identified as most important across all models.
        # This can potentially improve the model's performance by focusing on the most relevant information.
        ensemble.fit(self.X_test[self.top_features], self.y_test)
        
        # Predict the class labels for the test set using the ensemble model.
        predictions = ensemble.predict(self.X_test[self.top_features])
        
        # Print a classification report for the ensemble model, comparing its performance to the individual models.
        print(f"Classification Report (Ensemble):\n{classification_report(self.y_test, predictions)}")

    def evaluate_multiple_subsets(self, feature_subsets):
        # Initialize a dictionary to hold evaluation scores for different feature subsets.
        scores_by_subset = {}

        # Iterate over each subset of features defined in `feature_subsets`.
        for subset_name, features in feature_subsets.items():
            # Evaluate the model performance using the current subset of features.
            # This function trains the model on the subset and returns various performance metrics.
            scores = self.evaluate_subset(subset_name, features, self.X_train, self.X_test, self.y_train, self.y_test, feature_subsets)
            
            # Store the scores for the current subset by its name.
            scores_by_subset[subset_name] = scores

            # Plot the impact of the current feature subset on model performance.
            self.plot_subset_impact(subset_name, *scores)

            # An optional step to display the metrics for the current subset
            #self.display_metrics(*scores)

        # Return the collected scores for all subsets.
        return scores_by_subset

    def evaluate_subset_allfeatures(self):
        # Initialize a dictionary to store the scores when all features are used.
        scores_with = {}

        # Train the models defined in the class. This step assumes that `train_model` not only initializes but also tunes the models, making them ready for performance evaluation.
        self.train_model()

        # Iterate over each trained model to evaluate its performance using all available features.
        for model_name, model in self.models.items():
            # Print the name of the model currently being evaluated.
            print(model_name)

            # Initialize a dictionary to store ROC AUC related metrics for the current model.
            self.roc_auc[model_name] = {}

            # Fit the model on the training set and predict the test set.
            model.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)

            # Calculate various performance metrics including accuracy, precision, recall, and F1 score.
            accuracy = accuracy_score(self.y_test, y_pred)
            precision = precision_score(self.y_test, y_pred)
            recall = recall_score(self.y_test, y_pred)
            f1 = f1_score(self.y_test, y_pred)
            scores_with[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

            # For ROC AUC calculation, predict probabilities and compute the curve and area.
            y_pred_prob = model.predict_proba(self.X_test)[:, 1]
            fpr, tpr, thresholds = roc_curve(self.y_test, y_pred_prob, pos_label=1)
            roc_auc_area = roc_auc_score(self.y_test, y_pred_prob)

            # Store the ROC curve metrics and area under the curve (AUC) for the current model.
            self.roc_auc[model_name]['fpr'] = fpr
            self.roc_auc[model_name]['tpr'] = tpr
            self.roc_auc[model_name]['thresholds'] = thresholds
            self.roc_auc[model_name]['roc_auc'] = roc_auc_area

        # Return the scores for all models when all features are used.
        return scores_with


    def evaluate_subset(self, subset_name, features, X_train, X_test, y_train, y_test, feature_subsets):
        """
        Evaluate model performance using different configurations of feature subsets to understand
        the impact of certain features on model accuracy and other metrics.
        """
        # Initialize dictionaries to store scores for different feature configurations.
        scores_with_traditional = {}
        scores_with = {}
        scores_without = {}
        scores_subset_only = {}
        scores_subset_and_original = {}

        # Aggregate all unique features across all subsets for analysis.
        all_subset_features = set()
        for feature_subset in feature_subsets.values():
            all_subset_features.update(feature_subset)

        # Identify features present in the original dataset but not in any subset.
        original_minus_subset = [feature for feature in X_train.columns if feature not in all_subset_features]

        for model_name, model in self.models.items():

            # Evaluate using all features.
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            # Store accuracy, precision, recall, and F1 score.
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            scores_with[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

            # Evaluate using traditional features (original features minus subset features).
            model.fit(X_train[original_minus_subset], y_train)
            y_pred_original_minus_subset = model.predict(X_test[original_minus_subset])
            accuracy = accuracy_score(y_test, y_pred_original_minus_subset)
            precision = precision_score(y_test, y_pred_original_minus_subset)
            recall = recall_score(y_test, y_pred_original_minus_subset)
            f1 = f1_score(y_test, y_pred_original_minus_subset)
            scores_with_traditional[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

            # Evaluate without the specific subset of features.
            X_train_sub = X_train.drop(features, axis=1)
            X_test_sub = X_test.drop(features, axis=1)
            model.fit(X_train_sub, y_train)
            y_pred_sub = model.predict(X_test_sub)
            accuracy = accuracy_score(y_test, y_pred_sub)
            precision = precision_score(y_test, y_pred_sub)
            recall = recall_score(y_test, y_pred_sub)
            f1 = f1_score(y_test, y_pred_sub)
            scores_without[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

            # Evaluate with only the subset of features.
            model.fit(X_train[features], y_train)
            y_pred_subset = model.predict(X_test[features])
            accuracy = accuracy_score(y_test, y_pred_subset)
            precision = precision_score(y_test, y_pred_subset)
            recall = recall_score(y_test, y_pred_subset)
            f1 = f1_score(y_test, y_pred_subset)
            scores_subset_only[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

            # Evaluate with a combination of the subset and traditional features.
            combined_features = list (set(features + list(original_minus_subset)))
            model.fit(X_train[combined_features], y_train)
            y_pred_subset_and_original = model.predict(X_test[combined_features])
            accuracy = accuracy_score(y_test, y_pred_subset_and_original)
            precision = precision_score(y_test, y_pred_subset_and_original)
            recall = recall_score(y_test, y_pred_subset_and_original)
            f1 = f1_score(y_test, y_pred_subset_and_original)
            scores_subset_and_original[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}


            # Check for performance improvement
            #print(f"For {model_name}, Original Score: {scores_with[model_name]:.4f}, Without Subset: {scores_without[model_name]:.4f}, RFE Score: {scores_with_rfe[model_name]:.4f}, L1 Score: {scores_with_l1[model_name]:.4f}, Tree Score: {scores_with_tree[model_name]:.4f}")
            # Print a header for the evaluation metrics specific to the model and feature subset being analyzed.
            print(f"Evaluation Metrics for Model: {model_name} using '{subset_name}' subset of features")
            # Display the accuracy when using traditional features only.
            print(f"  - Accuracy with traditional features: {scores_with_traditional[model_name]}")
            # Display the accuracy when the model is trained with all available features.
            print(f"  - Accuracy with all features: {scores_with[model_name]}")
            # Display the accuracy when the model is trained with a combination of the evaluated subset and traditional features.
            # This shows whether integrating the subset with traditional features offers any performance improvement.
            print(f"  - Accuracy with '{subset_name}' subset + traditional features: {scores_subset_and_original[model_name]}")

        # Return the dictionaries containing scores for all configurations tested.
        # These scores include models trained with traditional features, all features, without the subset, with the subset only, and with a combination of the subset and traditional features.
        return scores_with_traditional, scores_with, scores_without, scores_subset_only, scores_subset_and_original


    def add_values_on_bars(self, ax, values, offset=0.01):
        """
        Annotate bar plots with their numerical values for better readability.

        Parameters:
        - ax: The matplotlib axes object where the bars are plotted.
        - values: A list of values corresponding to each bar to be annotated.
        - offset: A vertical offset to adjust the position of the annotation above the bar.
        """
        for i, v in enumerate(values):
            # Annotate each bar with its respective value, adjusting the position for visibility.
            # 'ha' and 'va' control the alignment of the text.
            ax.text(i, v + offset, f"{v:.2f}", fontsize=10, ha='center', va='bottom')


    def plot_subset_impact(self, subset_name, scores_with_traditional, scores_with, scores_without, scores_subset_only, scores_subset_and_original):
        """
        Visualize the impact of different feature subsets on model performance metrics.

        Parameters:
        - subset_name: Name of the feature subset being evaluated.
        - scores_*: Dictionaries containing scores for models trained with different feature configurations.
        """
        # Retrieve model names as they are consistent across all score dictionaries.
        model_names = list(scores_with_traditional.keys())
        print("Inside the Plot Subset impact Names")

        # Iterate over a set of metrics to create separate bar plots for each.
        for metric in ['Accuracy','Precision','Recall','F1 Score']:
            plt.figure(figsize=(20, 6))
            barWidth = 0.1  # Set the width of each bar.
            # Calculate the positions of each group of bars on the X axis.
            r1 = np.arange(len(model_names))
            r2 = [x + barWidth for x in r1]
            r3 = [x + barWidth for x in r2]
            r4 = [x + barWidth for x in r3]
            r5 = [x + barWidth for x in r4]

            '''
            r6 = [x + barWidth for x in r5]
            r7 = [x + barWidth for x in r6]
            r8 = [x + barWidth for x in r7]
            r9 = [x + barWidth for x in r8]
             '''
            # Plot bars for each feature configuration across all models for the current metric.
            y_values1 = [scores_with[item][metric] for item in scores_with]
            plt.bar(r1, y_values1, color='b', width=barWidth, edgecolor='grey', label='With All Features')
            y_values2 = [scores_with_traditional[item][metric] for item in scores_with_traditional]
            plt.bar(r2, y_values2, color='black', width=barWidth, edgecolor='grey', label='With Traditional Features Only')
            y_values3 = [scores_without[item][metric] for item in scores_without]
            plt.bar(r3, y_values3, color='r', width=barWidth, edgecolor='grey', label='Without Subset')
            y_values4 = [scores_subset_only[item][metric] for item in scores_subset_only]
            plt.bar(r4, y_values4, color='g', width=barWidth, edgecolor='grey', label='With Subset Only')
            y_values5 = [scores_subset_and_original[item][metric] for item in scores_subset_and_original]
            plt.bar(r5, y_values5, color='y', width=barWidth, edgecolor='grey', label='With Subset and Traditional')

            '''
            y_values6 = [scores_with_rfe[item]['Accuracy'] for item in scores_with_rfe]
            plt.bar(r6, y_values6, color='c', width=barWidth, edgecolor='grey', label='With RFE')
            y_values7 = [scores_with_l1[item]['Accuracy'] for item in scores_with_l1]
            plt.bar(r7, y_values7, color='m', width=barWidth, edgecolor='grey', label='With L1')
            y_values8 = [scores_with_tree[item]['Accuracy'] for item in scores_with_tree]
            plt.bar(r8, y_values8, color='orange', width=barWidth, edgecolor='grey', label='With Tree')
            y_values9 = [scores_with_anova[item]['Accuracy'] for item in scores_with_anova]
            plt.bar(r9, y_values9, color='purple', width=barWidth, edgecolor='grey', label='With ANOVA')
            '''
            # Adding numerical values on top of the bars for clarity.
            ax = plt.gca() 
            values_lists = [y_values1, y_values2, y_values3, y_values4, y_values5] # List of values for each group of bars.
            x_values = [r1, r2, r3, r4, r5] # Corresponding x positions for each group.

            # Debug prints to verify the values and positions.
            print(values_lists)
            print('x values:',x_values)
            # Loop through each group of bars to annotate them.
            for i in range(len(x_values)):
                print('In loop:',i)
                print(x_values[i])
                print(values_lists[i])
                # Annotate each bar with its value for better readability.
                for category, value in zip(x_values[i], values_lists[i]):
                    ax.text(category, value, "{:.2f}".format(value), ha='center', va='bottom')

            # Set labels for the x and y axes with specific font properties.
            plt.xlabel('Models', fontweight='bold', fontsize=16)
            plt.ylabel(f'{metric}', fontsize=16)
            # Title for the plot that includes the subset name and the method used for model evaluation.
            plt.title(f'Impact of {subset_name} Features on Model Performance [Method: {self.mode}]', fontsize=20)
            # Set x-ticks to be in the middle of each group of bars.
            plt.xticks([r + 4 * barWidth for r in range(len(r1))], model_names)  # Adjust the multiplier based on the number of bars

            #offset = 0.01  # Adjust this value for vertical placement
            #for values in values_lists:
            #    self.add_values_on_bars(plt.gca(), values, offset)

            plt.legend()
            plt.savefig(f"subset_impact_metric_{metric}_{subset_name}.png")
            #plt.show()
            plt.close()


    def display_metrics(self, scores_with_traditional, scores_with, scores_without, scores_subset_only, scores_subset_and_original):
        """
        Prints out the performance metrics for models trained with different feature configurations.

        Parameters:
        - scores_with_traditional: Dictionary containing model scores using traditional features.
        - scores_with: Dictionary containing model scores using all available features.
        - scores_without: Dictionary containing model scores excluding the specified feature subset.
        - scores_subset_only: Dictionary containing model scores using only the specified feature subset.
        - scores_subset_and_original: Dictionary containing model scores using both the specified subset and traditional features.
        """
        print("\nModel Performance Metrics:")
        for model_name in scores_with_traditional.keys():
            # For each model, print out the accuracy for different feature configurations.
            print(f"\nFor the model: {model_name}")
            print(f"  - Accuracy with traditional features: {scores_with_traditional[model_name]:.4f}")
            print(f"  - Accuracy with all features: {scores_with[model_name]:.4f}")
            print(f"  - Accuracy without the subset: {scores_without[model_name]:.4f}")
            print(f"  - Accuracy with only the subset: {scores_subset_only[model_name]:.4f}")
            print(f"  - Accuracy with the subset + original features: {scores_subset_and_original[model_name]:.4f}")


    def predict(self, data):
        """
        Predicts the outcome using an ensemble model composed of all the trained models.

        Parameters:
        - data: The dataset to predict outcomes for, expected to contain the top features.
        
        Returns:
        - The predicted outcomes based on the ensemble of models.
        """

        # Create an ensemble model using VotingClassifier with 'hard' voting strategy.
        # 'Hard' voting means that the predicted class label is the one that has received the most votes.
        ensemble = VotingClassifier(estimators=list(self.models.items()), voting='hard')
        # Predict the outcomes for the provided data using the ensemble model.
        # It uses the `top_features` determined during the model training phase for prediction.
        return ensemble.predict(data[self.top_features])

In [ ]:
# Instantiate the StrokePrediction class with a specified data path and the mode set to 'undersample'.
# The 'undersample' mode indicates that the class will balance the dataset by undersampling the majority class during the preprocessing phase to address class imbalance.
predictor = StrokePrediction(path,mode='undersample')
#X_train, X_test, y_train, y_test = predictor.preprocess_data()

# Train models using the undersampled dataset. This step involves fitting the models defined within the StrokePrediction class on the training data. 
predictor.train_model()

In [ ]:
# Evaluate the trained models on the test dataset to assess their performance.
predictor.evaluate_model()

In [ ]:
# Define a dictionary mapping thematic categories to lists of features. Each category represents a different aspect
# of the individuals' economic, neighborhood, personal, demographic, racial, ethnic, social, and mental health characteristics.
# This organization helps in analyzing the impact of various feature groups on the model's ability to predict strokes.
feature_subsets = {
    "Economic": ['Working','Working_Last_Week','Regular_Healthcare_Provider'],
    "Neighbourhood": [ 'Urban_Rural','Household_Ownership'],
    "Personal":['Problem_Handling_Ability','Day_to_Day_Demands_Ability', 'Perceived_Health', 'Perceived_Life_Stress'],
    "Demographic":['DHH_SEX', 'DHH_AGE', 'DHH_MS'],
    "Race": ['White','Chinese','South_Asian','Black','Filipino','Latin_America',
            'South_East_Asian','Arab','West_Asian','Japanese','Korean'],
    "Ethnicity": ['Canadian','French','English','German','Scottish','Irish','Italian',
            'Ukrainian','Dutch','Chinese_Ethnic','Jewish','Polish','South_Asian_Ethnic','Norwegian', 'Welsh','Swedish','Other','Metis','Inuit'],
    "Social":['Food_Security', 'Food_Affordability', 'Sense_of_Community_Belonging','Level_of_Education'],
    "Mental_Health":['Mood_Disorder','Anxiety_Disorder','Mental_Health_Consult_12mo']

    # Add more subsets as needed
}

# Evaluate the performance of models for each defined feature subset within the `feature_subsets` dictionary.
# This step involves training and testing the models on each subset of features and then aggregating the scores
# to understand which feature groups are most predictive of strokes. The method `evaluate_multiple_subsets` is
# handles the training, testing, and evaluation process, returning a dictionary of scores keyed by subset name.
scores_by_subset = predictor.evaluate_multiple_subsets(feature_subsets)

In [ ]:
# Evaluate models using all available features.
predictor.evaluate_subset_allfeatures()

In [ ]:
# Ensure matplotlib plots are displayed inline in the Jupyter notebook.
%matplotlib inline

# Prepare for a new figure to plot the ROC curves.
plt.close()

# Plot the ROC curve for each model. Each curve represents the model's performance in distinguishing between the positive and negative classes across different thresholds.
plt.plot(predictor.roc_auc['RandomForest']['fpr'], predictor.roc_auc['RandomForest']['tpr'], label='Random Forest (%.2f)'%predictor.roc_auc['RandomForest']['roc_auc'])
plt.plot(predictor.roc_auc['GradientBoosting']['fpr'], predictor.roc_auc['GradientBoosting']['tpr'], label='GradientBoosting (%.2f)'%predictor.roc_auc['GradientBoosting']['roc_auc'])
plt.plot(predictor.roc_auc['KNN']['fpr'], predictor.roc_auc['KNN']['tpr'], label='KNN (%.2f)'%predictor.roc_auc['KNN']['roc_auc'])
plt.plot(predictor.roc_auc['LGBM']['fpr'], predictor.roc_auc['LGBM']['tpr'], label='LGBM (%.2f)'%predictor.roc_auc['LGBM']['roc_auc'])
plt.plot(predictor.roc_auc['NeuralNetwork']['fpr'], predictor.roc_auc['NeuralNetwork']['tpr'], label='NeuralNetwork (%.2f)'%predictor.roc_auc['NeuralNetwork']['roc_auc'])
plt.plot(predictor.roc_auc['CatBoost']['fpr'], predictor.roc_auc['CatBoost']['tpr'], label='CatBoost (%.2f)'%predictor.roc_auc['CatBoost']['roc_auc'])
plt.plot(predictor.roc_auc['XGBoost']['fpr'], predictor.roc_auc['XGBoost']['tpr'], label='XGBoost (%.2f)'%predictor.roc_auc['XGBoost']['roc_auc'])

# Plot the line for a random classifier as a baseline comparison. The diagonal dashed line represents a model that predicts classes in a completely random manner, with AUC = 0.5.
plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')
# Set the labels and title for the ROC plot. 
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (All features)[Method: Undersampling]')
plt.legend(loc="lower right")
plt.show()
